In [1]:
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
from dash_iconify import DashIconify
import dash_mantine_components as dmc

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# import data
kreise_df = pd.read_csv('/home/jan/Uni/DS-Project/modules/dashboard/test/apps/assets/kreise_df.csv')
# rename and drop columns
kreise_df = kreise_df.drop(columns=['Unnamed: 0']).rename(columns={'NAME_1': 'Bundesland', 'NAME_2': 'Region', 'NAME_3': 'Landkreis', 'ENGTYPE_3': 'Land_Stadt'})

In [3]:
%%script false --no-raise-error

def flatten_list(lst):
    flattened_list = []
    for item in lst:
        if type(item) == np.ndarray:
            flattened_list += [x for x in item.tolist()]
        else:
            flattened_list.append(item)
    return flattened_list

def parent_finder(children):
    df = kreise_df
    children_list = df[children].unique().tolist()
    parent_list = []
    parent_column = df.columns.get_loc(children) - 1
    for child in children_list:
        parent_value = df.loc[df[children] == child].iloc[:, parent_column].unique()
        parent_list.append(parent_value)
    return flatten_list(parent_list)

def value_finder(typus):
    df = kreise_df
    value_list = []
    unit_list = df[typus].unique().tolist()
    for unit in unit_list:
        value_value = round(np.mean(df.loc[df[typus] == unit]['avg_Score']), 1)
        value_list.append(value_value)
    return value_list

In [4]:
%%script false --no-raise-error

# create suburst vectors
labels = kreise_df.Bundesland.unique().tolist() + kreise_df.Region.unique().tolist() + kreise_df.Landkreis.unique().tolist()
parents = [''] * len(kreise_df.Bundesland.unique()) + parent_finder('Region') + parent_finder('Landkreis')
values = value_finder('Bundesland') + value_finder('Region') + value_finder('Landkreis')

In [5]:
# create sunburst plot
Ohren_sunburst = px.sunburst(kreise_df, path=['Bundesland', 'Region', 'Landkreis'], values='avg_Score')
Ohren_sunburst = Ohren_sunburst.update_traces(
    insidetextorientation='radial',
    hovertemplate = '<b>%{label}</b> <br>Score: %{value}')

In [6]:
app = Dash(__name__, external_stylesheets = [dbc.themes.LITERA])

In [7]:
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "26rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and add some padding.
CONTENT_STYLE = {
    "margin-left": "28rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

In [8]:
sidebar = html.Div([
    dbc.Row([
      html.H1('Dashboard für PV-Ausbau in "Ohren"'),
      html.Hr(),
      html.P('In this research project we seek to identify the potential of photovolatic power in highway turnoffs across Germany.')
    ]),
    dmc.Tabs([
              dmc.TabsList([
            dmc.Tab("Einleitung", value = "einleitung"),
            dmc.Tab("Motivation", value = "motivation"),
            dmc.Tab("Aufbau", value = "aufbau"),
            dmc.Tab("Fazit", value = "fazit"),
        ]),
    ]),
    dcc.Markdown('''
        | Mitwirkende | Matrikelnummer |
        | ---         | ---:           |
        |Yvette Brody | 1234 |
        |Marvin Hoberg| 1234 |
        |Felix Schulz | 1234 |
        |Jan Besler   |5629079|
    '''),
    # Links und Erklärungen
    dbc.Row([
      dbc.Col(["GitHub Repo",
        DashIconify(icon="bi:github"),
        #href="https://github.com/felixschulz385/ds_project",
      ]),
      dbc.Col(["How it started",
        DashIconify(icon="bi:newspaper"),
        #href="https://www.badische-zeitung.de/ob-palmer-will-solaranlagen-auf-freien-stellen-neben-bundesstrassen--211487616.html",
      ]),
      html.Hr(),
      html.P('This project is being developed as part of the \
              Data Science in Business and Economics Master degree course at the University of Tübingen.')
    ])
], style=SIDEBAR_STYLE)

In [9]:
# images
import base64
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/apps/assets/economic_model.jpeg", "rb") as file:
    economic_model_img = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/apps/assets/brandenburg_driveways.jpg", "rb") as file:
    BB_ohren = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
    
    # carousel pictures
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/apps/assets/Lustnauer_Ohren_vorher.jpg", "rb") as file:
    carousel_1 = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/apps/assets/Lustnauer_Ohren_danach.jpg", "rb") as file:
    carousel_2 = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))

In [10]:
import dash_mantine_components as dmc
from dash import Output, Input, html, callback

content = html.Div(
    children=[
        dmc.Accordion(
            id="accordion-simple",
            value="customization",
            children=[
                dmc.AccordionItem(
                    [
                        dmc.AccordionControl("Customization"),
                        dmc.AccordionPanel(
                                                dbc.Row([
                        dcc.Markdown('''
                        Das 'Road to Renewables' Projekt hilft bisher ungenutzte Flächen für den Ausbau von Erneuerbaren Energien
                        aufzuzeigen. Ziel ist es interessierten Bürger:innen und Gemeinden visuell ansprechend darzulegen, wo in
                        Deutschland Potenziale vorhanden sind. Die betrachteten Flächen, werden so ausgewählt, dass es keine
                        Interessenskonflikte über Verwendungsmöglichkeiten geben soll.
                            ''')])
                        ),
                    ],
                    value="customization",
                ),
                dmc.AccordionItem(
                    [
                        dmc.AccordionControl("Flexibility"),
                        dmc.AccordionPanel(
                            "Configure temp appearance and behavior with vast amount of settings or overwrite any "
                            "part of component styles "
                        ),
                    ],
                    value="flexibility",
                ),
            ],
        ),
        dmc.Text(id="accordion-state", mt=10),
    ]
)


@callback(Output("accordion-state", "children"), Input("accordion-simple", "value"))
def show_state(value):
    return value

In [11]:
content = html.Div([
    dmc.Tabs([
        dmc.TabsList([
            dmc.Tab("Einleitung", value = "einleitung"),
            dmc.Tab("Motivation", value = "motivation"),
            dmc.Tab("Aufbau", value = "aufbau"),
            dmc.Tab("Fazit", value = "fazit"),
        ], grow=True),

        # Einleitung
    dmc.TabsPanel(
        dbc.Container([
            dbc.Row([
                html.H4('Einleitung & Motivation')
        ]),
        dmc.Accordion(
            value="einleitung_accordion",
            children=[
            dmc.AccordionItem([
                dmc.AccordionControl("Einleitung"),
                dmc.AccordionPanel(
                    dbc.Row([
                        dcc.Markdown('''
                        Das 'Road to Renewables' Projekt hilft bisher ungenutzte Flächen für den Ausbau von Erneuerbaren Energien
                        aufzuzeigen. Ziel ist es interessierten Bürger:innen und Gemeinden visuell ansprechend darzulegen, wo in
                        Deutschland Potenziale vorhanden sind. Die betrachteten Flächen, werden so ausgewählt, dass es keine
                        Interessenskonflikte über Verwendungsmöglichkeiten geben soll.
                            ''')])
                    )],  value="einleitung_accordion"),
            dmc.AccordionItem([
                dmc.AccordionControl("Motivation"),
                dmc.AccordionPanel([
                    dbc.Row([
                        html.H4('Motivation')]),
                    dbc.Row([
                        dbc.Col([
                        dcc.Markdown('''
                        Der globale Klimawandel hat die deutsche Energiewirtschaft in Aufruhr versetzt. 
                        Nach Jahren der starken Abhängigkeit von Kohle und Gas (im Jahr 2010 wurden 25% des deutschen Energiemixes 
                        des deutschen Energiemixes auf diese Weise bezogen ([BDEW, 2022](https://www.bdew.de/energie/bruttostromerzeugung-seit-2010/)), 
                        haben die Klimazusagen die Politiker gezwungen, auf erneuerbare Energien umzusteigen 
                        ([Bundesregierung, 2022](https://www.bundesregierung.de/breg-en/issues/climate-action/government-climate-policy-1779414)). 
                            ''')], width=7),
                        dbc.Col([
                            dbc.Card([
                            dbc.CardImg(src = BB_ohren, top=True),
                            dbc.CardBody(html.P("Beispielbild der potenziellen Flächen.", className="card-text")),
                            ],style={"height": "60vh", "width": "60vh"})
                        ], width=4)
                    ]),
                ], value="Motivation")
            ])
        ]),


        ]), value = 'einleitung'
    ),

        # Motivation
    dmc.TabsPanel(
        dbc.Container([
           dbc.Row([
            html.H4('Motivation')
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Markdown('''
                Der globale Klimawandel hat die deutsche Energiewirtschaft in Aufruhr versetzt. 
                Nach Jahren der starken Abhängigkeit von Kohle und Gas (im Jahr 2010 wurden 25% des deutschen Energiemixes 
                des deutschen Energiemixes auf diese Weise bezogen ([BDEW, 2022](https://www.bdew.de/energie/bruttostromerzeugung-seit-2010/)), 
                haben die Klimazusagen die Politiker gezwungen, auf erneuerbare Energien umzusteigen 
                ([Bundesregierung, 2022](https://www.bundesregierung.de/breg-en/issues/climate-action/government-climate-policy-1779414)). 
            ''')
            ], width=7),
            dbc.Col([
                dbc.Card([
                dbc.CardImg(src = BB_ohren, top=True),
                dbc.CardBody(
                    html.P("Beispielbild der potenziellen Flächen.", className="card-text")),
            ],style={"height": "60vh", "width": "60vh"})
            ], width=4)
        ])
        ]), value = "motivation"
    ),
        
        # Aufbau
    dmc.TabsPanel(
        dbc.Container([
        dbc.Row([
            html.H4('Aufbau')
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Markdown('''
                Our project seeks to provide a comprehensive overview on the potential in equiping highway turnoffs across Germany with photovoltaics.
                Using state-of-the-art GIS software and Machine Learning techniques we identify turnoffs and evaluate their suitability.
                Beginning with a limited scope on the German state of Brandenburg, we plan to extend our models to cover the entire nation's road network.
            '''),
            ], width=7),
            dbc.Col([
            dbc.Card([
                html.Img(src = economic_model_img),
                dbc.CardBody(
                    html.P("Structure of economic model.", className="card-text")),
            ],style={"height": "60vh", "width": "60vh"})
            ], width=4),
        ])]), value= "aufbau"
    ),
        
    # Sunburst Plot
    dmc.TabsPanel(
        dbc.Container([
            dbc.Row([
            html.H3('Überblick Landkreise', id = "ueberblick")
        ]),
        # Sunburst Plot
        dbc.Row([dbc.Col([
            dcc.Graph(figure = Ohren_sunburst, responsive=True, style = {'height': '80vh', 'width': '80vh'})
            ], class_name = "d-flex justify-content-center")
        ])
        ]), value = "fazit"
    )
    ],
    color = "yellow",
    orientation = "horizontal",
    value = "einleitung"
    )

], style=CONTENT_STYLE)

TypeError: The `dash_mantine_components.AccordionPanel` component (version 0.11.1) received an unexpected keyword argument: `value`
Allowed arguments: children, className, id, m, mb, ml, mr, mt, mx, my, p, pb, pl, pr, pt, px, py, style, styles, sx, unstyled

keine Matrikelnummern
Namen Yvette ändern
alle Interviews aufnehmen
Piot Projket deutschalnd

In [11]:
app.layout = html.Div([sidebar, content])

# run app
if __name__ == '__main__':
    app.run_server(port = 8010)

Dash is running on http://127.0.0.1:8010/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8010
Press CTRL+C to quit
127.0.0.1 - - [15/Feb/2023 15:09:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2023 15:09:32] "GET /assets/dashExtensions_default.js?m=1676385345.0 HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2023 15:09:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2023 15:09:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2023 15:09:33] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2023 15:09:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2023 15:09:33] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -


# Test Bereich